In [1]:
import tensorflow as tf
import numpy as np
import os

import data_utils



In [2]:
#1479175091
#1479582035
#1479176649
checkpoint_dir = os.path.join(os.path.abspath(os.path.curdir), "runs", "1512866911", "checkpoints")

# Shared Parameters
# ==================================================
embedding_dim = 64
batch_size = 54
hidden_size = 100
num_classes = 5
num_epochs = 10
learning_rate = 0.001


In [3]:
# Load Data
# ==================================================
test_filename = "pnp-test.txt"

x_test, y_test, seq_lens_test, vocab_dict \
    = data_utils.load_data_and_labels(test_filename, train=False)
test_data = list(zip(x_test, y_test, seq_lens_test))

print(vocab_dict)


{';': 119, 'ü': 75, '-': 34, 'ñ': 89, 'ä': 74, 'R': 37, 'S': 19, '$': 120, 'F': 35, ':': 53, '&': 63, 'U': 55, 'ý': 140, ')': 78, 'Ô': 112, '`': 143, 'ã': 90, 'ì': 124, 'P': 25, 'O': 46, 'Â': 123, 'ö': 72, 'J': 45, 'Ð': 136, 'C': 17, '[': 142, 'Ë': 111, 'd': 13, 'Ü': 121, 'L': 29, 'á': 79, 'y': 20, 'D': 27, 'Ä': 126, 'Q': 69, 'æ': 113, 'w': 40, '0': 50, 'k': 31, 'Ñ': 118, 'G': 38, 'K': 48, '7': 71, '÷': 138, 'g': 16, 'ð': 109, '~': 0, 'õ': 107, 'Ø': 125, 'Ö': 122, 'à': 85, 'þ': 141, '°': 133, '@': 145, 'û': 97, '6': 66, 'ï': 128, 'Ó': 101, 'Ì': 132, 'Z': 64, 'ó': 80, '¿': 129, 'l': 9, 'h': 14, 'Ç': 110, 'é': 62, 'É': 95, 'V': 49, 's': 10, '(': 76, 'â': 92, 'E': 39, 'x': 43, 'T': 21, 'Õ': 115, '2': 51, 'N': 41, 'è': 77, '3': 56, '5': 58, ']': 137, 'í': 81, 'f': 30, 'ê': 102, '_': 139, 'c': 12, '"': 100, '*': 98, 'A': 22, 'ß': 96, '4': 57, 'b': 28, 'å': 84, 'I': 24, 't': 8, 'ÿ': 130, 'Ò': 103, 'Ê': 131, 'q': 59, 'r': 6, '9': 65, 'o': 5, '/': 70, 'î': 116, 'X': 67, 'n': 4, 'ú': 108, 'j': 

In [4]:
# Evaluation
# ==================================================
checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)

graph = tf.Graph()
with graph.as_default():
    sess = tf.Session()

    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        input_y = graph.get_operation_by_name("input_y").outputs[0]
        seq_lens = graph.get_operation_by_name("seq_lens").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("prediction/predictions").outputs[0]
        prediction_probs = graph.get_operation_by_name("prediction/prediction_probs").outputs[0]
        correct_vector = graph.get_operation_by_name("prediction/correct_vector").outputs[0]

        # Collect the predictions here
        all_predictions = []
        all_labels = []

        # Generate batches for one epoch
        batches = data_utils.batch_iter(test_data, batch_size=batch_size, num_epochs=1, shuffle=False)

        probs = []

        for batch in batches:
            x_batch = batch[:,0]
            y_batch = batch[:,1]
            seq_lens_batch = batch[:,2]

            x_batch = data_utils.pad(x_batch, seq_lens_batch)

            batch_predictions, correct, batch_probs = sess.run([predictions, correct_vector, prediction_probs], \
            #batch_predictions = sess.run([predictions], \
                {input_x: x_batch, input_y: y_batch, seq_lens: seq_lens_batch})
            #print(batch_probs)
            correct += np.sum(correct)
            #print(y_batch)
            all_labels = np.concatenate([all_labels, y_batch]) #np.concatenate([all_labels, [np.argmax(x) for x in y_batch]])
            all_predictions = np.concatenate([all_predictions, batch_predictions])

            for i in range(len(y_batch)):
                probs.append(batch_probs[i][y_batch[i]])

# Print accuracy if y_test is defined; [np.argmax(x) for x in all_labels]
correct_predictions = np.sum(all_predictions == np.array(all_labels))
print(correct_predictions)
print("Total number of test examples: {}".format(len(seq_lens_test)))
print("Accuracy: {:g}".format(correct_predictions/float(len(seq_lens_test))))

labels = ["drug", "person", "place", "movie", "company"]

INFO:tensorflow:Restoring parameters from /home/nbuser/library/runs/1512866911/checkpoints/model-4001
2357
Total number of test examples: 2862
Accuracy: 0.82355


In [5]:
all_predictions

array([ 0.,  0.,  0., ...,  0.,  4.,  4.])

In [6]:
# Write predictions
test_filename = "pnp-test.txt"

file_path = os.path.join(os.path.abspath(os.path.curdir), test_filename)
f = open(file_path, 'r', encoding = "ISO-8859-1")
data = list(f.readlines())
f.close()
data = [s.strip().split() for s in data]

proper_nouns_strings = [" ".join(d[1:]) for d in data]

f = open("output2.txt", 'w', encoding = "ISO-8859-1")
for i in range(len(proper_nouns_strings)):
    f.write("Example: "+proper_nouns_strings[i]+" guess="+str(labels[int(all_predictions[i])])+" gold= confidence="+str(probs[i])+"\n")
f.close()
